In [ ]:
import unicodedata
from sklearn.feature_extraction.text import TfidfVectorizer
from activecleanSVM.imdb import IMDB

plot_filepath, genre_filepath, yahoo_filepath = "data/plot.list", "data/imdb-genres.list", \
                                                "data/ydata-ymovies-movie-content-descr-v1_0.txt"

print("Processing data...")
imdb_data_processor = IMDB(plot_filepath, genre_filepath, yahoo_filepath)
X, Y = imdb_data_processor.get_comedy_and_horror_movie_data()
dirty_data = (X, Y)
dirty_indices = list(range(len(Y)))
clean_indices = []
indices = [dirty_indices, clean_indices]
total_labels = []

vectorizer = TfidfVectorizer(stop_words="english", max_features=50000)
text = [unicodedata.normalize("NFKD", i[0] + " " + i[1]).encode("ascii", "ignore") for i in X]
vectorizer.fit_transform(text)
print("Done processing")

In [ ]:
import ast
from sklearn import svm
from activecleanSVM.activecleanprocessor import ActiveCleanProcessor

# User defined functions
def data_to_features(data, vectorizer):
    text = [
        unicodedata.normalize("NFKD", i[0] + " " + i[1]).encode(
            "ascii", "ignore"
        )
        for i in data
    ]

    return vectorizer.transform(text)

def process_cleaned_df(df):
    def convertStrToList(text):
        lst = ast.literal_eval(text)
        assert type(lst) == list
        return lst

    df["Genres"] = df["Genres"].apply(lambda x: convertStrToList(x))
    return df

# User Defined Variables
own_filepath = "C:\\Users\\isabe\\OneDrive\\Desktop\\ActiveCleanFiles\\"
num_records_to_clean = 10
batch_size = 5
step_size = 0.1  # learning_rate
model = svm.LinearSVC()

print("Initialising ActiveClean...")
ActiveClean = ActiveCleanProcessor(model, dirty_data, indices,
                                   batch_size, own_filepath, step_size,
                                   vectorizer, data_to_features, process_cleaned_df)

ActiveClean.start(num_records_to_clean)

In [ ]:
print("Starting retraining...")
updated_clf, X_full, Y_full = ActiveClean.runNextIteration()

#Run this cell until you reach your desired number of records cleaned
#Clean training and test data seperate using this process